In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import requests
import pickle
import sys
import re
import csv

You have not imported the dnspython module


In [2]:
crawled_liste = []

In [3]:
class Article:
    def __init__(self, title, lead, datum, zeit, author, text, url):
        self.title = title
        self.lead = lead
        self.datum = datum
        self.zeit = zeit
        self.author = author
        self.text = text
        self.url = url

In [4]:
def inhalter():    
    #main
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    #segments
    try:
        title = soup.find(class_=re.compile("title nau-t-typography-display-1")).get_text()
        title = title.strip()
    except AttributeError:
        title = ""

    ## lead (as it is not optional to have one)
    try:
        lead = soup.find(class_=re.compile("lead nau-t-typography-display-3")).get_text()
        lead = lead.strip()
    except AttributeError:
        lead = ""

    # datum (is not always given)
    try:
        datum = soup.find(class_=re.compile("date")).get_text()
        zeit = datum[-5:]
        zeit = zeit.strip()
        datum = datum.replace("Am ", "")
        datum = datum[:-8]
        datum = datum.strip()
    except AttributeError:
        datum = ""
        zeit = ""

    #author
    try:
        author = soup.find(class_=re.compile("author-name ng-star-inserted")).get_text()
    except AttributeError:
        author = ""
    if "Beitrag von" in author:
        author = author.replace("Beitrag von ", "")
        author = author.strip()
    
    
    text = str()
    blocktext = soup.find_all(class_=re.compile("nau-t-typography"))
    for teil in blocktext:
        texty = teil.get_text()
        texty = texty.replace('\t','')
        texty = texty.replace('\n','')
        text += " " + texty
    if "Mehr zum Thema" in text:
        p = text.index("Mehr zum Thema")
        text = text[:p]
        text = text.strip()
    if ".  " in text:
        text = text.replace(".  ", ". ")
    ding_url = page
    
    crawled = Article(title, lead, datum, zeit, author, text, ding_url)
    crawled_liste.append(crawled)

In [5]:
driver = webdriver.Firefox()
driver.implicitly_wait(30)
driver.maximize_window()

In [6]:
start_seite = "https://www.nau.ch/politik/regional/prozess-gegen-pierre-maudet-in-genf-eroffnet-65871426"
driver.get(start_seite)

cookies = pickle.load(open("nau_cookies.pkl", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)
    
driver.refresh()

In [7]:
with open("nau_output.bin", "rb") as data:
    ueb_pages = pickle.load(data)
    

In [8]:
for page in ueb_pages[9460:]:
    time.sleep(2)
    try:
        driver.get(page)
        inhalter()
    except TimeoutException:
        print("TimeOut")
        pass    

In [10]:
with open('nau_art.csv', 'w', newline='', encoding="utf8") as csvfile:
    blogwriter = csv.writer(csvfile, delimiter='|', quotechar='"')


    for article in crawled_liste:
        blogwriter.writerow( [article.title, article.lead, article.datum, article.zeit, article.author, article.text, article.url] )

In [ ]:
driver.close()

# Quick fix
In case one needs to interrupt crawling and wants to continue later where the crawler stopped, run the cell below and update cell 7, first line from: <br>
*"for page in ueb_pages:"* <br>
to : <br>
*"for page in ueb_pages[go_on:]:"*

This method also works in case of uncatched errors or interrupted kernels, provided kernel has not been restarted yet. In case kernel will be restarted, please note down the value of go_on or pickle dump the value beforehand as the value of go_on will not be stored automatically.
